In [1]:
import re
import nltk
import string
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import gensim
from gensim.parsing.preprocessing import remove_stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
df = pd.read_csv("./tweet_data/all_tweets.csv")
df.head()

,id,date,tweet,username,nlikes,nreplies,nretweets
0,1.26E+18,2020-05-13 13:46,"$20 #BITCOIN #GIVEAWAY????\n\nTo celebrate partner @shndofficial releasing their trading platform, 1??? lucky winner will win $20 #BTC\n????Follow @shndxofficial & @CircuitSociety\n????Retweet-Lik...",CircuitSociety,103.0,63.0,229.0
1,1.26E+18,2020-05-13 13:27,"25,000 Sats #BTC #Giveaway\n????RT & follow @Bitsahara\n????Join chat: http://t.me/bitsahara?? (optional)\n\n2 Winners in 24 Hours!??????\n#Bitcoin #LightningNetwork #SSWB pic.twitter.com/62T0CLCTaF",Bitsahara,201.0,97.0,275.0
2,1.26E+18,2020-05-13 12:58,??????@HooExchange listed @AtomicWallet token #AWC\ntrading pair $AWC/ $USDT is now available for both #ERC20 & #bep2 token.\nCHECK IT HERE??? https://twitter.com/AtomicWallet/status/126054977972...,SmartCryptoNew1,57.0,6.0,179.0
3,1.26E+18,2020-05-13 12:51,"""How to buy BITCOIN #BTC on OKEx Exchange"" And\n\n""How to transfer token from your OKEx Funding Account to Spot Trading Account on OKEx Echange""\n\n#TradewithOKEx @OKEx\n \n Together let's trade ...",taratitat_l0di,74.0,10.0,215.0
4,1.26E+18,2020-05-13 12:25,????#Follow me for Instant airdrop!\nEverybody rewarded by our super fast airdrop. \n\n???? Join me on telegram group for More uplates: http://telegram.me/allcryptokings??\n#Giveaway #crypto #800...,cryptoenews,45.0,2.0,171.0


In [3]:
df['cleaned_data']=df['tweet'].fillna('').astype("string")

In [4]:
df['cleaned_data']

0        $20 #BITCOIN #GIVEAWAY????

To celebrate partner @shndofficial releasing their trading platform, 1??? lucky winner will win $20 #BTC
????Follow @shndxofficial & @CircuitSociety
????Retweet-Like
??...
1             25,000 Sats #BTC #Giveaway
????RT & follow @Bitsahara
????Join chat:  http://t.me/bitsahara?? (optional)

2 Winners in 24 Hours!??????
#Bitcoin #LightningNetwork #SSWB pic.twitter.com/62T0CLCTaF
2        ??????@HooExchange  listed @AtomicWallet token #AWC
trading pair $AWC/ $USDT is now available for both #ERC20 & #bep2 token.
CHECK IT HERE??? https://twitter.com/AtomicWallet/status/12605497797230...
3        "How to buy BITCOIN #BTC  on OKEx Exchange" And

"How to transfer token from your OKEx Funding Account to Spot Trading Account on OKEx Echange"

#TradewithOKEx @OKEx
 
 Together let's trade on OKE...
4        ????#Follow me for Instant airdrop!
Everybody rewarded by our super fast airdrop. 

???? Join me on telegram group for More uplates:  http://telegram.me/al

In [5]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'',text)
df['cleaned_data'] = df['cleaned_data'].apply(lambda text : remove_urls(text))

In [6]:
def remove_pic(text):
    url_pattern = re.compile(r'pic\.twitter\.com\S+')
    return url_pattern.sub(r'',text)

df['cleaned_data'] = df['cleaned_data'].apply(lambda text : remove_pic(text))

In [7]:
df

,id,date,tweet,username,nlikes,nreplies,nretweets,cleaned_data
0,1.26E+18,2020-05-13 13:46,"$20 #BITCOIN #GIVEAWAY????\n\nTo celebrate partner @shndofficial releasing their trading platform, 1??? lucky winner will win $20 #BTC\n????Follow @shndxofficial & @CircuitSociety\n????Retweet-Lik...",CircuitSociety,103.0,63.0,229.0,"$20 #BITCOIN #GIVEAWAY????\n\nTo celebrate partner @shndofficial releasing their trading platform, 1??? lucky winner will win $20 #BTC\n????Follow @shndxofficial & @CircuitSociety\n????Retweet-Lik..."
1,1.26E+18,2020-05-13 13:27,"25,000 Sats #BTC #Giveaway\n????RT & follow @Bitsahara\n????Join chat: http://t.me/bitsahara?? (optional)\n\n2 Winners in 24 Hours!??????\n#Bitcoin #LightningNetwork #SSWB pic.twitter.com/62T0CLCTaF",Bitsahara,201.0,97.0,275.0,"25,000 Sats #BTC #Giveaway\n????RT & follow @Bitsahara\n????Join chat: (optional)\n\n2 Winners in 24 Hours!??????\n#Bitcoin #LightningNetwork #SSWB"
2,1.26E+18,2020-05-13 12:58,??????@HooExchange listed @AtomicWallet token #AWC\ntrading pair $AWC/ $USDT is now available for both #ERC20 & #bep2 token.\nCHECK IT HERE??? https://twitter.com/AtomicWallet/status/126054977972...,SmartCryptoNew1,57.0,6.0,179.0,??????@HooExchange listed @AtomicWallet token #AWC\ntrading pair $AWC/ $USDT is now available for both #ERC20 & #bep2 token.\nCHECK IT HERE??? \n#AWC #listed #crypto #SCN1\n@coinkit_ mon 10 250 BTC
3,1.26E+18,2020-05-13 12:51,"""How to buy BITCOIN #BTC on OKEx Exchange"" And\n\n""How to transfer token from your OKEx Funding Account to Spot Trading Account on OKEx Echange""\n\n#TradewithOKEx @OKEx\n \n Together let's trade ...",taratitat_l0di,74.0,10.0,215.0,"""How to buy BITCOIN #BTC on OKEx Exchange"" And\n\n""How to transfer token from your OKEx Funding Account to Spot Trading Account on OKEx Echange""\n\n#TradewithOKEx @OKEx\n \n Together let's trade ..."
4,1.26E+18,2020-05-13 12:25,????#Follow me for Instant airdrop!\nEverybody rewarded by our super fast airdrop. \n\n???? Join me on telegram group for More uplates: http://telegram.me/allcryptokings??\n#Giveaway #crypto #800...,cryptoenews,45.0,2.0,171.0,????#Follow me for Instant airdrop!\nEverybody rewarded by our super fast airdrop. \n\n???? Join me on telegram group for More uplates: \n#Giveaway #crypto #800cc #telegram #telegramgroup\n@coink...
...,...,...,...,...,...,...,...,...
41483,9.08E+17,2017-09-12 20:12,$STRAT will be added @binance_2017 on 14/09/2017 at 4am UTC #blockchain #bitcoin #dotnet #stratis pic.twitter.com/iOa1IDgXEV,stratisplatform,359.0,10.0,212.0,$STRAT will be added @binance_2017 on 14/09/2017 at 4am UTC #blockchain #bitcoin #dotnet #stratis
41484,9.08E+17,2017-09-12 14:27,STRAT trading on Binance! \n https://binance.zendesk.com/hc/en-us/articles/115001522771???????,binance,345.0,23.0,191.0,STRAT trading on Binance! \n
41485,9.07E+17,2017-09-09 12:57,Binance Will List OMG Market\n https://binance.zendesk.com/hc/en-us/articles/115001495491???????,binance,675.0,74.0,329.0,Binance Will List OMG Market\n
41486,9.04E+17,2017-09-01 10:04,We would love you all to participate in the @binance_2017 community vote and vote for $STRAT #stratis https://www.binance.com/vote.html?? #bitcoin $BTC pic.twitter.com/UaexPz1r4Q,stratisplatform,276.0,23.0,179.0,We would love you all to participate in the @binance_2017 community vote and vote for $STRAT #stratis #bitcoin $BTC


In [8]:
df['cleaned_data'] = df['cleaned_data'].str.lower()
df.head()

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [9]:
Punct_to_remove = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('','',Punct_to_remove))

df['cleaned_data'] = df['cleaned_data'].apply(lambda text : remove_punctuation(text))
df.head()

from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [10]:
STOPWORDS = set(stopwords.words('english'))

def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df['cleaned_data'] = df['cleaned_data'].apply(lambda text : remove_stopwords(text))
df.head()

,id,date,tweet,username,nlikes,nreplies,nretweets,cleaned_data
0,1.26E+18,2020-05-13 13:46,"$20 #BITCOIN #GIVEAWAY????\n\nTo celebrate partner @shndofficial releasing their trading platform, 1??? lucky winner will win $20 #BTC\n????Follow @shndxofficial & @CircuitSociety\n????Retweet-Lik...",CircuitSociety,103.0,63.0,229.0,20 bitcoin giveaway celebrate partner shndofficial releasing trading platform 1 lucky winner win 20 btc follow shndxofficial circuitsociety retweetlike tag 3 friends crct listed ends 517 coinkit m...
1,1.26E+18,2020-05-13 13:27,"25,000 Sats #BTC #Giveaway\n????RT & follow @Bitsahara\n????Join chat: http://t.me/bitsahara?? (optional)\n\n2 Winners in 24 Hours!??????\n#Bitcoin #LightningNetwork #SSWB pic.twitter.com/62T0CLCTaF",Bitsahara,201.0,97.0,275.0,25000 sats btc giveaway rt follow bitsahara join chat optional 2 winners 24 hours bitcoin lightningnetwork sswb
2,1.26E+18,2020-05-13 12:58,??????@HooExchange listed @AtomicWallet token #AWC\ntrading pair $AWC/ $USDT is now available for both #ERC20 & #bep2 token.\nCHECK IT HERE??? https://twitter.com/AtomicWallet/status/126054977972...,SmartCryptoNew1,57.0,6.0,179.0,hooexchange listed atomicwallet token awc trading pair awc usdt available erc20 bep2 token check awc listed crypto scn1 coinkit mon 10 250 btc
3,1.26E+18,2020-05-13 12:51,"""How to buy BITCOIN #BTC on OKEx Exchange"" And\n\n""How to transfer token from your OKEx Funding Account to Spot Trading Account on OKEx Echange""\n\n#TradewithOKEx @OKEx\n \n Together let's trade ...",taratitat_l0di,74.0,10.0,215.0,buy bitcoin btc okex exchange transfer token okex funding account spot trading account okex echange tradewithokex okex together lets trade okex link cctipio airdrop 0011 eth 200
4,1.26E+18,2020-05-13 12:25,????#Follow me for Instant airdrop!\nEverybody rewarded by our super fast airdrop. \n\n???? Join me on telegram group for More uplates: http://telegram.me/allcryptokings??\n#Giveaway #crypto #800...,cryptoenews,45.0,2.0,171.0,follow instant airdrop everybody rewarded super fast airdrop join telegram group uplates giveaway crypto 800cc telegram telegramgroup coinkit mon 20 200 btc


In [11]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jinny\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

df['cleaned_data'] = df['cleaned_data'].apply(lambda text : lemmatize_words(text))
df.head()

,id,date,tweet,username,nlikes,nreplies,nretweets,cleaned_data
0,1.26E+18,2020-05-13 13:46,"$20 #BITCOIN #GIVEAWAY????\n\nTo celebrate partner @shndofficial releasing their trading platform, 1??? lucky winner will win $20 #BTC\n????Follow @shndxofficial & @CircuitSociety\n????Retweet-Lik...",CircuitSociety,103.0,63.0,229.0,20 bitcoin giveaway celebrate partner shndofficial releasing trading platform 1 lucky winner win 20 btc follow shndxofficial circuitsociety retweetlike tag 3 friend crct listed end 517 coinkit mon...
1,1.26E+18,2020-05-13 13:27,"25,000 Sats #BTC #Giveaway\n????RT & follow @Bitsahara\n????Join chat: http://t.me/bitsahara?? (optional)\n\n2 Winners in 24 Hours!??????\n#Bitcoin #LightningNetwork #SSWB pic.twitter.com/62T0CLCTaF",Bitsahara,201.0,97.0,275.0,25000 sat btc giveaway rt follow bitsahara join chat optional 2 winner 24 hour bitcoin lightningnetwork sswb
2,1.26E+18,2020-05-13 12:58,??????@HooExchange listed @AtomicWallet token #AWC\ntrading pair $AWC/ $USDT is now available for both #ERC20 & #bep2 token.\nCHECK IT HERE??? https://twitter.com/AtomicWallet/status/126054977972...,SmartCryptoNew1,57.0,6.0,179.0,hooexchange listed atomicwallet token awc trading pair awc usdt available erc20 bep2 token check awc listed crypto scn1 coinkit mon 10 250 btc
3,1.26E+18,2020-05-13 12:51,"""How to buy BITCOIN #BTC on OKEx Exchange"" And\n\n""How to transfer token from your OKEx Funding Account to Spot Trading Account on OKEx Echange""\n\n#TradewithOKEx @OKEx\n \n Together let's trade ...",taratitat_l0di,74.0,10.0,215.0,buy bitcoin btc okex exchange transfer token okex funding account spot trading account okex echange tradewithokex okex together let trade okex link cctipio airdrop 0011 eth 200
4,1.26E+18,2020-05-13 12:25,????#Follow me for Instant airdrop!\nEverybody rewarded by our super fast airdrop. \n\n???? Join me on telegram group for More uplates: http://telegram.me/allcryptokings??\n#Giveaway #crypto #800...,cryptoenews,45.0,2.0,171.0,follow instant airdrop everybody rewarded super fast airdrop join telegram group uplates giveaway crypto 800cc telegram telegramgroup coinkit mon 20 200 btc


In [13]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\jinny\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [15]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

In [16]:
lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN,"V":wordnet.VERB,"J":wordnet.ADJ,"R":wordnet.ADV}
def lemmatize_word(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word,wordnet_map.get(pos[0],wordnet.NOUN)) for word , pos in 
                    pos_tagged_text])
df['cleaned_data'] = df['cleaned_data'].apply(lambda text : lemmatize_word(text))
df.head()

,id,date,tweet,username,nlikes,nreplies,nretweets,cleaned_data
0,1.26E+18,2020-05-13 13:46,"$20 #BITCOIN #GIVEAWAY????\n\nTo celebrate partner @shndofficial releasing their trading platform, 1??? lucky winner will win $20 #BTC\n????Follow @shndxofficial & @CircuitSociety\n????Retweet-Lik...",CircuitSociety,103.0,63.0,229.0,20 bitcoin giveaway celebrate partner shndofficial release trading platform 1 lucky winner win 20 btc follow shndxofficial circuitsociety retweetlike tag 3 friend crct list end 517 coinkit mon 50 ...
1,1.26E+18,2020-05-13 13:27,"25,000 Sats #BTC #Giveaway\n????RT & follow @Bitsahara\n????Join chat: http://t.me/bitsahara?? (optional)\n\n2 Winners in 24 Hours!??????\n#Bitcoin #LightningNetwork #SSWB pic.twitter.com/62T0CLCTaF",Bitsahara,201.0,97.0,275.0,25000 sat btc giveaway rt follow bitsahara join chat optional 2 winner 24 hour bitcoin lightningnetwork sswb
2,1.26E+18,2020-05-13 12:58,??????@HooExchange listed @AtomicWallet token #AWC\ntrading pair $AWC/ $USDT is now available for both #ERC20 & #bep2 token.\nCHECK IT HERE??? https://twitter.com/AtomicWallet/status/126054977972...,SmartCryptoNew1,57.0,6.0,179.0,hooexchange list atomicwallet token awc trading pair awc usdt available erc20 bep2 token check awc list crypto scn1 coinkit mon 10 250 btc
3,1.26E+18,2020-05-13 12:51,"""How to buy BITCOIN #BTC on OKEx Exchange"" And\n\n""How to transfer token from your OKEx Funding Account to Spot Trading Account on OKEx Echange""\n\n#TradewithOKEx @OKEx\n \n Together let's trade ...",taratitat_l0di,74.0,10.0,215.0,buy bitcoin btc okex exchange transfer token okex funding account spot trading account okex echange tradewithokex okex together let trade okex link cctipio airdrop 0011 eth 200
4,1.26E+18,2020-05-13 12:25,????#Follow me for Instant airdrop!\nEverybody rewarded by our super fast airdrop. \n\n???? Join me on telegram group for More uplates: http://telegram.me/allcryptokings??\n#Giveaway #crypto #800...,cryptoenews,45.0,2.0,171.0,follow instant airdrop everybody reward super fast airdrop join telegram group uplates giveaway crypto 800cc telegram telegramgroup coinkit mon 20 200 btc


In [47]:
pip install spellchecker

  Created wheel for spellchecker: filename=spellchecker-0.4-py3-none-any.whl size=3966522 sha256=0e43c6fd4c6eda618637009062d1656b85b7d6e5b7f00fe645b4cb604b07b8b4
  Stored in directory: c:\users\jinny\appdata\local\pip\cache\wheels\58\e9\48\b82b733a7a0a9cc52ed239ccf082ff33e2fbda71670ddd3349
  Created wheel for inexactsearch: filename=inexactsearch-1.0.2-py3-none-any.whl size=7118 sha256=951ead4da4d16ff90a560c270b9d5f091789964a881de67298179647f224038b
  Stored in directory: c:\users\jinny\appdata\local\pip\cache\wheels\3c\f3\5d\77e8983e86af1c3c62b80d76de8020f560ce6367f1b2763b40
  Created wheel for soundex: filename=soundex-1.1.3-py3-none-any.whl size=8867 sha256=09804e0869049cbee944d2cbd29d34bbaba8e38ccaf40197fdcd2c9d5f8c168e
  Stored in directory: c:\users\jinny\appdata\local\pip\cache\wheels\6b\22\0f\0ac590ac1dc0fa60f57151bf39f1fae86a9978a1d54b48e197
  Created wheel for silpa-common: filename=silpa_common-0.3-py3-none-any.whl size=8455 sha256=a38c5560d7c22971751d76b151519b4c02e7ccb5f5b

In [ ]:
df1 = pd.read_csv("cleaned_tweets1.csv")
df1.head()

In [17]:
df["cleaned_data"]= df["cleaned_data"].str.replace('\d+', '')

In [149]:
df1

,Unnamed: 0,id,date,tweet,username,nlikes,nreplies,nretweets,cleaned_data
0,0,1.26E+18,2020-05-13 13:46,"$20 #BITCOIN #GIVEAWAY????\n\nTo celebrate partner @shndofficial releasing their trading platform, 1??? lucky winner will win $20 #BTC\n????Follow @shndxofficial & @CircuitSociety\n????Retweet-Lik...",CircuitSociety,103.0,63.0,229.0,bitcoin giveaway celebrate partner shndofficial release trading platform lucky winner win btc follow shndxofficial circuitsociety retweetlike tag friend crct list end coinkit mon btc
1,1,1.26E+18,2020-05-13 13:27,"25,000 Sats #BTC #Giveaway\n????RT & follow @Bitsahara\n????Join chat: http://t.me/bitsahara?? (optional)\n\n2 Winners in 24 Hours!??????\n#Bitcoin #LightningNetwork #SSWB pic.twitter.com/62T0CLCTaF",Bitsahara,201.0,97.0,275.0,sat btc giveaway rt follow bitsahara join chat optional winner hour bitcoin lightningnetwork sswb
2,2,1.26E+18,2020-05-13 12:58,??????@HooExchange listed @AtomicWallet token #AWC\ntrading pair $AWC/ $USDT is now available for both #ERC20 & #bep2 token.\nCHECK IT HERE??? https://twitter.com/AtomicWallet/status/126054977972...,SmartCryptoNew1,57.0,6.0,179.0,hooexchange list atomicwallet token awc trading pair awc usdt available erc bep token check awc list crypto scn coinkit mon btc
3,3,1.26E+18,2020-05-13 12:51,"""How to buy BITCOIN #BTC on OKEx Exchange"" And\n\n""How to transfer token from your OKEx Funding Account to Spot Trading Account on OKEx Echange""\n\n#TradewithOKEx @OKEx\n \n Together let's trade ...",taratitat_l0di,74.0,10.0,215.0,buy bitcoin btc okex exchange transfer token okex funding account spot trading account okex echange tradewithokex okex together let trade okex link cctipio airdrop eth
4,4,1.26E+18,2020-05-13 12:25,????#Follow me for Instant airdrop!\nEverybody rewarded by our super fast airdrop. \n\n???? Join me on telegram group for More uplates: http://telegram.me/allcryptokings??\n#Giveaway #crypto #800...,cryptoenews,45.0,2.0,171.0,follow instant airdrop everybody reward super fast airdrop join telegram group uplates giveaway crypto cc telegram telegramgroup coinkit mon btc
...,...,...,...,...,...,...,...,...,...
41483,41483,9.08E+17,2017-09-12 20:12,$STRAT will be added @binance_2017 on 14/09/2017 at 4am UTC #blockchain #bitcoin #dotnet #stratis pic.twitter.com/iOa1IDgXEV,stratisplatform,359.0,10.0,212.0,strat add binance am utc blockchain bitcoin dotnet stratis
41484,41484,9.08E+17,2017-09-12 14:27,STRAT trading on Binance! \n https://binance.zendesk.com/hc/en-us/articles/115001522771???????,binance,345.0,23.0,191.0,strat trading binance
41485,41485,9.07E+17,2017-09-09 12:57,Binance Will List OMG Market\n https://binance.zendesk.com/hc/en-us/articles/115001495491???????,binance,675.0,74.0,329.0,binance list omg market
41486,41486,9.04E+17,2017-09-01 10:04,We would love you all to participate in the @binance_2017 community vote and vote for $STRAT #stratis https://www.binance.com/vote.html?? #bitcoin $BTC pic.twitter.com/UaexPz1r4Q,stratisplatform,276.0,23.0,179.0,would love participate binance community vote vote strat stratis bitcoin btc


In [18]:
df1.to_csv('all_cleaned_tweets.csv')

NameError: name 'df1' is not defined